In [ ]:
pip install openai-clip torch torchvision pillow pinecone tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.5 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=0767ee5a52acf6110375709ae06fefce044995c475d06ab4bffa3914dcf55a14
  Stored in directory: /root/.cache/pip/wheels/ab/49/bc/c2342e8e14878210ba4825cf314a53f2570f6fb18b91fce3cf
Successfully built openai-clip
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [ ]:
!pip install tensorflow.keras

In [ ]:
import torch
import clip
from PIL import Image
from tqdm import tqdm
from pinecone import Pinecone, ServerlessSpec

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 118MiB/s]


In [ ]:
import os

image_folder = "/content/drive/MyDrive/Dataset/Normal-skin"
image_vectors = []

for filename in tqdm(os.listdir(image_folder)):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder, filename)
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)


        with torch.no_grad():
            image_features = model.encode_image(image)

        image_features /= image_features.norm(dim=-1, keepdim=True)
        vector = image_features.cpu().numpy().flatten().tolist()

        image_vectors.append({
            "id": filename,
            "values": vector,
            "metadata": {"Disease": "Normal"}
        })


In [ ]:
# Batch upsert
for i in range(0, len(image_vectors), 100):
    batch = image_vectors[i:i+100]
    index.upsert(vectors=batch)

print(f"✅ Uploaded {len(image_vectors)} image embeddings to Pinecone")


✅ Uploaded 2618 image embeddings to Pinecone


In [ ]:
pc = Pinecone(api_key="pcsk_xEske_CL1K1Kxm8Zu2ncNFPaVW9TKroYJHYJn7KYb6Vtug66GUF5q8mFPLN9JbpWkGhgP")

index_name = "clip-skd"

# Create new index if it doesn’t exist
if index_name not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=512,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)


In [ ]:

#if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
#image_path = os.path.join(image_folder, filename)
def RAG(image,index):


  with torch.no_grad():
      image_features = model.encode_image(image)

  image_features /= image_features.norm(dim=-1, keepdim=True)
  query_vector = image_features.cpu().numpy().flatten().tolist()
  results = index.query(
      vector=query_vector,
      top_k=1,
      include_metadata=True
  )
  results = results["matches"]
  return results

In [ ]:
pc = Pinecone(api_key="pcsk_xEske_CL1K1Kxm8Zu2ncNFPaVW9TKroYJHYJn7KYb6Vtug66GUF5q8mFPLN9JbpWkGhgP")
image_path = "/content/OIP.webp"
image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
index_name = "clip-wri"
index = pc.Index(index_name)
RAG(image,index)

[{'id': 'wrinkles128.png',
  'metadata': {'filename': 'wrinkles128.png'},
  'score': 0.942565918,
  'values': []}]

In [ ]:
++9996from langgraph.graph import START, END, StateGraph
builder = StateGraph(GenerateAnalystsState)
builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", human_feedback)
builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback", should_continue, ["create_analysts", END])

In [ ]:
import joblib
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/aagp.keras")
image_path = "/content/drive/MyDrive/Dataset/Normal-skin/07637_png_jpg.rf.aa7de7bd68a2a7ce3a650c1ae2fc1b01.jpg"
from tensorflow.keras.preprocessing.image import load_img
image = Image.open(image_path)
img = image.resize((128, 128), Image.Resampling.LANCZOS)
img = np.array(img)
pred = model.predict(img)
pred

TypeError: argument 1 must be sequence of length 2, not 3

In [ ]:
from pydantic import BaseModel
from typing import Optional,List
class State(BaseModel):
  age: int
  gender: str
  bauman_type: str
  skin_disease: Optional[str] = None
  meds: Optional[List[str]] = []

In [ ]:
builder = StateGraph(State)
builder.add_node("", RAG, attributes={'image':,'index':})
builder.add_node("", RAG, attributes={'image':,'index':})
builder.add_node("", RAG, attributes={'image':,'index':})
builder.add_node("", RAG, attributes={'image':,'index':})
builder.add_node("", RAG, attributes={'image':,'index':})
builder.add_node("", human_feedback)
builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedb]ack")
builder.add_conditional_edges("human_feedback", should_continue, ["create_analysts", END])

In [1]:
!pip install tavily

  Preparing metadata (setup.py) ... done
  Created wheel for tavily: filename=tavily-1.1.0-py3-none-any.whl size=6128 sha256=724fe288c487d1e242b8ff05915b00c365abd144239c2903cc1be1a509469ecc
  Stored in directory: /root/.cache/pip/wheels/a7/67/b7/9aec4851724de28ac2bc34ff10b042af43d7f2dd1552e5906e
Successfully built tavily


In [26]:
from tavily import TavilyClient
client = TavilyClient(api_key="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")
response = client.search("Please suggest a loreal skin care product of age 27 and skin type is oily, sensitive, pigmented, wrinkle. Give only lorealparisusa websites")
for i in response['results']:
  print(i)

url=response['results'][0]['url']

{'url': 'https://www.lorealparisusa.com/beauty-magazine/skin-care/anti-aging/best-anti-aging-skin-care-products', 'title': "11 of Our Best Anti-Aging Skincare Products for 2025 | L'Oréal Paris", 'content': 'The best anti-aging skincare products can help address those unwanted concerns while keeping your skin feeling balanced, hydrated, and healthy. Ahead, we’re sharing a look at 11 of our top skincare products for aging skin, from cleansers and moisturizers to targeted brightening serums and more. Products with hyaluronic acid are ideal for plumping and hydrating the skin, while retinol is touted for its ability to help smooth the look of fine lines and wrinkles. How often you should use anti-aging skincare products depends on the formula and your skin type. However, if you’re looking for a basic regimen to get you started, we recommend cleansing your face (twice daily), applying an anti-aging serum, and smoothing on a moisturizer designed for your skin type and concerns.', 'score': 0.

In [13]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 20.4 MB/s eta 0:00:00


In [23]:
import requests
url='https://www.lorealparisusa.com/skin-care/oily-skin'
data  = requests.get(url).text
from bs4 import BeautifulSoup
soup = BeautifulSoup(data,"html")
requests.get(url)

<Response [200]>

In [29]:
table = soup.find_all("article")#, attrs={"alt": "071249135655_T1"})
#print(table)
# f=[]
# do=[]
# a=table.find_all('tr')
# for row in a: # in html table row is represented by the tag <tr>
#     # Get all columns in each row.
#     d=row.find_all('td')
#     try:
#         f.append(d[0].string)
#         do.append(d[1].string)
#     except:
#         pass
# da={'Date':f,"Revenue":do}
# tesla_revenue=pd.DataFrame(da)
# tesla_revenue["Revenue"]
table

[<article class="filterGrid__item">
 <div class="filterGrid__itemWrapper">
 </div>
 </article>,
 <article class="filterGrid__item">
 <div class="filterGrid__itemWrapper">
 </div>
 </article>,
 <article class="filterGrid__item">
 <div class="filterGrid__itemWrapper">
 </div>
 </article>,
 <article class="filterGrid__item">
 <div class="filterGrid__itemWrapper">
 </div>
 </article>,
 <article class="filterGrid__item">
 <div class="filterGrid__itemWrapper">
 </div>
 </article>,
 <article class="filterGrid__item">
 <div class="filterGrid__itemWrapper">
 </div>
 </article>,
 <article class="filterGrid__item">
 <div class="filterGrid__itemWrapper">
 </div>
 </article>,
 <article class="filterGrid__item">
 <div class="filterGrid__itemWrapper">
 </div>
 </article>,
 <article class="filterGrid__item">
 <div class="filterGrid__itemWrapper">
 </div>
 </article>,
 <article class="filterGrid__item">
 <div class="filterGrid__itemWrapper">
 </div>
 </article>,
 <article class="filterGrid__item">
 <di

In [15]:
from langchain_tavily import TavilySearch
tavily_search = TavilySearch(max_results=3)
data = tavily_search.invoke({"query": state['question']})
search_docs = data.get("results", data)

ModuleNotFoundError: No module named 'langchain_tavily'